In [ ]:
from horsewords import fimfarchive

# This folder should contain the unpacked fimfarchive data.
CACHE_PATH = '/bulk/data/fimfarchive/fimfarchive - Mar 1, 2024'

ff = fimfarchive.Fimfarchive(CACHE_PATH)

In [ ]:
from datasets import Dataset

def datapoints():
    for story_id, story in ff.stories_by_id.items():
        try:
            chapters = ff.get_cached_chapters(story_id)
        except:
            continue

        chapter_metadata = sorted(story['chapters'], key=lambda x: x['chapter_number'])
        for chapter, metadata in zip(chapters, chapter_metadata):
            yield {
                'author': story['author']['name'],
                'story': story_id,
                'chapter': metadata['chapter_number'],
                'story_tags': ','.join([x['name'] for x in story['tags']]),
                'story_title': story['title'],
                'story_blurb': story['short_description'],
                'story_status': story['completion_status'],
                'story_likes': story['num_likes'],
                'story_dislikes': story['num_dislikes'],
                'story_description': story['description_html'],
                'story_rating': story['content_rating'],
                'created': metadata['date_published'],
                'updated': metadata['date_modified'],
                'views': metadata['num_views'],
                'text': chapter
            }

dataset = Dataset.from_generator(datapoints)
dataset.push_to_hub("synthbot/fimfarchive")